# Obliczanie zwrotu z inwestycji


W tej części projektu, na podstawie sygnałów generwanych przez poprzedni kod pythona obliczone zostaną dzienne stopy zwrotu z rozważanych strategii

In [1]:
### BIBLIOTEKI ###
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### ŚCIEŻKI ##
file_path1 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/rates_data/rates_data.xlsx"
file_path2 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/prices_cleaned.xlsx"
file_path3 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/GPW capitalization USD mies.xlsx"
file_path4 = "C:/Users/olgas/OneDrive/Documents/GitHub/Hight-turnover-effect-GPW/zestawienie nazw.xlsx"



## Wczytywanie danych


Sygnały są zapisane w 15 strategiach (każda w osobnym arkuszu w pliku excel). Odczytuje wszytskiie do słownika ramek danych


In [2]:
prices = pd.read_excel(file_path2)

prices.iloc[:, 1:] = prices.iloc[:, 1:].applymap(lambda x: pd.to_numeric(x, errors='coerce'))

capitalization = pd.read_excel(file_path3)

accuracy = pd.read_excel(file_path4)


sheets_dict = pd.read_excel(file_path1, sheet_name=None)

sygnals = {} # Pusty słownik do zappisywania 
for sheet_name, dataframe in sheets_dict.items():
    sygnals[sheet_name] = dataframe

# Odwoływanie sie do konkretnych ramek danych -> sygnals['rates_10_1']

C:\Users\olgas\AppData\Local\Temp\ipykernel_19352\322302912.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  prices.iloc[:, 1:] = prices.iloc[:, 1:].applymap(lambda x: pd.to_numeric(x, errors='coerce'))


## Dzienne stopy zwrotu 

In [3]:
def daily_returns(df):
    result = pd.DataFrame(columns=df.columns)  # Nowa ramka danych do przechowywania wyników

    for col in df.columns:
        if col == 'Date':
            
            result['Date'] = df['Date'][1:].reset_index(drop=True)
        else:
            # Oblicz dzienne stopy zwrotu
            rates_values = []  # Pusta lista na dzienne stopy zwrotu
            for i in range(len(df) - 1):
                rate = (df[col].iloc[i + 1] - df[col].iloc[i]) / df[col].iloc[i]
                rates_values.append(rate)  # Dodaj wynik do listy

            result[col] = pd.Series(rates_values).reset_index(drop=True)

    return result

daily_returns_df = daily_returns(prices)

## Ujednolicenie kolumn 


W badaniu potrzeba ujednolicić kolumny w ramkach danych z kapitalizacją i cenami i sygnałami


In [6]:
columns_to_remove = accuracy.loc[accuracy['czy zgodnosc'].isnull(), 'Profesora'].tolist() # Kolumny których nie ma w mojej tabeli do usuniecia 

capitalizationv2 = capitalization.drop(columns=columns_to_remove) # usunięcie wartości których ja u siebie nie mam 

capitalizationv3 = capitalizationv2[prices.columns] # ustawienie w tej samej kolejności kolumny



## Okres inwestycji - 1 dzień 


W przypadku okresu sprzedaży równego 1 dzień, nie trzeba liczyć średnich dziennych stóp zwrotu z uwzględnieniem tego okresu 


In [ ]:
def one_day(sygnal_df, prices_df, price, kap_min, kap_df, daily_returns):

    
    # Upewnij się, że wszystkie daty są w formacie datetime
    sygnal_df['Year'] = pd.to_datetime(sygnal_df['Date']).dt.year
    sygnal_df['Month'] = pd.to_datetime(sygnal_df['Date']).dt.month
    kap_df['Year'] = pd.to_datetime(kap_df['Date']).dt.year
    kap_df['Month'] = pd.to_datetime(kap_df['Date']).dt.month

    # Dołącz kapitalizację na podstawie roku i miesiąca
    kap_df = kap_df.drop(columns=['Date'])  # Usuń oryginalną kolumnę Date, aby uniknąć konfliktów
    merged_df = pd.merge(
        sygnal_df,
        kap_df,
        on=['Year', 'Month'],
        how='left'
    )

    sygnal_df = sygnal_df.drop(columns=['Year', 'Month'])

    # Filtrowanie ramek
    filtered_kap = [col for col in merged_df.columns if col.endswith('_y')]
    filtered_kap = merged_df[filtered_kap]
    filtered_prices = prices_df.loc[sygnal_df.index]

    # Pusta ramka na wyniki
    result = pd.DataFrame(columns=['Date', 'Mean', 'Count'])


    # Iteracja po wierszach
    for i in range(len(sygnal_df)):
        row_sum = 0
        count = 0

        for col in sygnal_df.columns:
            if col == 'Date':
                result.loc[i, 'Date'] = sygnal_df.loc[i, 'Date']
            else:
                if not pd.isna(sygnal_df[col].iloc[i]):

                    # Sprawdzamy, czy kolumna istnieje w filtered_prices
                    if col in filtered_prices.columns:
                        # Pobieramy odpowiednią wartość kapitalizacji dla konkretnego wiersza
                        kap_value = filtered_kap.loc[i, col + '_y']

                        # Sprawdzamy warunki ceny i kapitalizacji
                        if (filtered_prices[col].iloc[i] > price and
                                kap_min <= kap_value and pd.notna(kap_value)):
                            row_sum += daily_returns[col].iloc[i]
                            count += 1
                    else:
                        print(f"Kolumna {col} nie istnieje w filtered_prices, pomijam wiersz {i}")

        # Obliczanie średniej i zapis wyników
        mean = row_sum / count if count > 0 else 0
        result.loc[i, 'Mean'] = mean
        result.loc[i, 'Count'] = count

    return result


In [7]:
results_50_1_v2 = one_day(sygnals['rates_50_1'], prices, 0, 0, capitalizationv2)

In [8]:
d1 = sygnals['rates_50_1']

# Teścik


In [ ]:
# Spółki powyżej cen 1 zł około 
results_10_1_v1 = one_day(sygnals['rates_10_1'], prices, 0.25, 0, capitalizationv2)
results_20_1_v1 = one_day(sygnals['rates_20_1'], prices, 0.25, 0, capitalizationv2)
results_50_1_v1 = one_day(sygnals['rates_50_1'], prices, 0.25, 0, capitalizationv2)
# Spółki powyżej cen 2 zł około 
results_10_1_v2 = one_day(sygnals['rates_10_1'], prices, 0.5, 0, capitalizationv2)
results_20_1_v2 = one_day(sygnals['rates_20_1'], prices, 0.5, 0, capitalizationv2)
results_50_1_v2 = one_day(sygnals['rates_50_1'], prices, 0.5, 0, capitalizationv2)
# Kap większa niż 1000000
results_10_1_v3 = one_day(sygnals['rates_10_1'], prices, 0.25, 1000000, capitalizationv2)
results_20_1_v3 = one_day(sygnals['rates_20_1'], prices, 0.25, 1000000, capitalizationv2)
results_50_1_v3 = one_day(sygnals['rates_50_1'], prices, 0.25, 1000000, capitalizationv2)
# Kap wieksze niż 2000000
results_10_1_v4 = one_day(sygnals['rates_10_1'], prices, 0.25, 2000000, capitalizationv2)
results_20_1_v4 = one_day(sygnals['rates_20_1'], prices, 0.25, 2000000, capitalizationv2)
results_50_1_v4 = one_day(sygnals['rates_50_1'], prices, 0.25, 2000000, capitalizationv2)
# Kap wieksze niz 5000000
results_10_1_v5 = one_day(sygnals['rates_10_1'], prices, 0.25, 5000000, capitalizationv2)
results_20_1_v5 = one_day(sygnals['rates_20_1'], prices, 0.25, 5000000, capitalizationv2)
results_50_1_v5 = one_day(sygnals['rates_50_1'], prices, 0.25, 5000000, capitalizationv2)
# Kap wieksze niż 10000000
results_10_1_v3 = one_day(sygnals['rates_10_1'], prices, 0.25, 10000000, capitalizationv2)
results_20_1_v3 = one_day(sygnals['rates_20_1'], prices, 0.25, 10000000, capitalizationv2)
results_50_1_v3 = one_day(sygnals['rates_50_1'], prices, 0.25, 10000000, capitalizationv2)


# Okres inwestycji, 5 dni

In [ ]:
def five_day(sygnal_df, prices_df, price, kap_min, kap_df, daily_returns):

    
    # Upewnij się, że wszystkie daty są w formacie datetime
    sygnal_df['Year'] = pd.to_datetime(sygnal_df['Date']).dt.year
    sygnal_df['Month'] = pd.to_datetime(sygnal_df['Date']).dt.month
    kap_df['Year'] = pd.to_datetime(kap_df['Date']).dt.year
    kap_df['Month'] = pd.to_datetime(kap_df['Date']).dt.month

    # Dołącz kapitalizację na podstawie roku i miesiąca
    kap_df = kap_df.drop(columns=['Date'])  # Usuń oryginalną kolumnę Date, aby uniknąć konfliktów
    merged_df = pd.merge(
        sygnal_df,
        kap_df,
        on=['Year', 'Month'],
        how='left'
    )

    sygnal_df = sygnal_df.drop(columns=['Year', 'Month'])

    # Filtrowanie ramek
    filtered_kap = [col for col in merged_df.columns if col.endswith('_y')]
    filtered_kap = merged_df[filtered_kap]
    filtered_prices = prices_df.loc[sygnal_df.index]

    # Pusta ramka na wyniki
    result = pd.DataFrame(columns=['Date', 'Mean', 'Count'])
    mid_results = pd.DataFrame (columns = ['Date', 's1', "s2", 's3','s4', 's5', 's1_c', "s2_c", 's3_c','s4_c', 's5_c'] )
    mid_results['Date'] = sygnal_df.loc[1:, 'Date' ]

    print(mid_results)
    # Iteracja po wierszach
    for i in range(4):

        for j in range(i, len(sygnal_df)):

            
            count = 0
            cols = [] # lista na nazwy kolumn ( nowa w każdej iteracji )

            for col in sygnal_df.columns:
                if col != 'Date':
                    if not pd.isna(sygnal_df[col].iloc[j]):

                        # Sprawdzamy, czy kolumna istnieje w filtered_prices
                        if col in filtered_prices.columns:
                            # Pobieramy odpowiednią wartość kapitalizacji dla konkretnego wiersza
                            kap_value = filtered_kap.loc[i, col + '_y']

                            # Sprawdzamy warunki ceny i kapitalizacji
                            if (filtered_prices[col].iloc[i] > price and
                                    kap_min <= kap_value and pd.notna(kap_value)):
                                cols.append(col)
                                count += 1
                        else:
                            print(f"Kolumna {col} nie istnieje w filtered_prices, pomijam wiersz {i}")
                
            column_name = f's{i+1}' 
            counter_column = f's{i+1}_c'


            for k in range(4) :
                # Obliczanie średniej i zapis wyników
                print(mid_results)
                print(j)
                print(i)
                print(column_name)
                print(counter_column)
                
                mean = daily_returns.loc[j+k,cols].sum() / count if count > 0 else 0
                print(mean)
                mid_results.loc[j+k, column_name] = mean
                mid_results.loc[j+k, counter_column] = count
        j += 5 


    result['Date'] = mid_results['Date']
    result['Mean'] = mid_results[['s1', 's2', 's3', 's4', 's5']].mean(axis=1, skipna=True)
    result['Count'] = mid_results[['s1_c', 's2_c', 's3_c', 's4_c', 's5_c']].sum(axis=1, skipna=True)
    return result 

In [ ]:
five_day(sygnals['rates_10_5'], prices, 0, 0, capitalizationv2, daily_returns_df)

# Skumulowane stopy zwrotu 


In [ ]:
cumulative_return = (1 + pd.Series(returns)).prod() - 1